In [0]:
#Conduct testing of one url to grab, and combine into "Bronze" Layer
testUrl=''  #input one test url to grab base schema


res = requests.get(testUrl)

# Save the content to a temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix=".xml") as temp_file:
    temp_file.write(res.content)
    temp_file_path = temp_file.name

# Move the temporary file to DBFS
dbfs_path = f"/dbfs/tmp/{temp_file.name.split('/')[-1]}"
shutil.move(temp_file_path, dbfs_path)

# Load the XML file from DBFS
df = (spark.read
      .format('xml')
      .options(rowTag='ClinicalDocument')
      .option('excludeAttribute', 'true')  # Exclude attributes to avoid column name conflicts
      .load(f"dbfs:/tmp/{temp_file.name.split('/')[-1]}"))

# Rename the conflicting column if necessary
df = df.withColumnRenamed('_value', '_value_renamed')

display(df)


inferred_schema = df.schema
print(inferred_schema.json())

# Saving the schema
dbutils.fs.put("/path/to/schema.json", inferred_schema.json())

# Loading the schema
schema_path = "/path/to/schema.json"
inferred_schema = StructType.fromJson(json.loads(dbutils.fs.head(schema_path)))

# Reading the XML file with the enforced schema
#df_with_schema = spark.read.format("xml").option("rowTag", "ClinicalDocument").schema(inferred_schema).load("/path/to/xmlfile")